In [18]:
import xarray as xr
import numpy as np
from pathlib import Path

In [7]:
data_path = Path("data/sat1/stage_data.nc")
stage_data = xr.load_dataset(data_path)

In [22]:
specific_labels = stage_data.sel(participant="0021", epochs=1).labels.to_numpy()
indices = np.where(specific_labels[:-1] != specific_labels[1:])[0] + 1
# Convert labels: ['','','','','1]

In [44]:
label_info.shape

(25, 200, 199)

In [65]:
label_info = stage_data.labels.to_numpy()
indices = np.where(label_info[:, :, :-1] != label_info[:, :, 1:])
# Add one when using

In [125]:
label_info[0, 4, :]

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'confirmation', 'confirmation',
       'confirmation', 'confirmation', 'confirmation', 'confirmation',
       'confirmation', 'confirmation', 'confirmation', 'confirmation',
       'confirmation', 'confirmation', 'confirmation', 'confirmation',
       'confirmation', 'confirmati

In [122]:
indices.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [137]:
# TODO: Has potential, find out which axes to split and how
split_arr = np.split(label_info, indices[2] + 1, axis=2)

In [113]:
print(len(split_arr))

11399


In [144]:
split_arr[2][0, 4, :]

array(['decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision'],
      dtype=object)

In [80]:
# indices
print(indices[0].shape)
print(indices[1].shape)
print(indices[2].shape)
for i, j, k in zip(indices[0], indices[1], indices[2]):
    print(label_info[i, j, k])
# Split based on indices? For CNN:
# Select participants before (stage_data.sel(participant=...))
# Then

(11398,)
(11398,)
(11398,)

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmatio

In [30]:
specific_labels[indices[0]: indices[1]]

array(['encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding'], dtype=object)